# Meta ML System Design: Data & Labels

**Target Level:** Meta E5 → E6  
**Core Skill:** Understanding where labels come from and their biases  
**Interview Focus:** Implicit feedback, label quality, data collection strategies

---

## Why This Matters

At Meta, **you rarely have clean labels.** Almost all ML systems learn from implicit user behavior:

- Clicks, not "this was relevant"
- Watch time, not "I enjoyed this"
- Hides, not "this was bad"

Understanding the gap between **what you observe** and **what you want to learn** is crucial for building systems that actually work.

---

## 1. Label Sources at Meta Scale

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           LABEL SOURCE HIERARCHY                            │
└─────────────────────────────────────────────────────────────────────────────┘

EXPLICIT LABELS (Rare, High Quality)
├── User ratings (1-5 stars)
├── Surveys ("Was this helpful?")
├── Human annotation (labelers)
└── User reports (spam, harmful content)
         │
         │  ↓ Decreasing quality, increasing volume
         ▼
IMPLICIT LABELS (Common, Noisy)
├── Strong signals
│   ├── Purchase / conversion
│   ├── Share to friends
│   ├── Save for later
│   └── Long dwell time (>30s)
│
├── Medium signals
│   ├── Like / reaction
│   ├── Comment
│   ├── Follow author
│   └── Complete video
│
├── Weak signals
│   ├── Click
│   ├── Scroll past slowly
│   └── Short dwell time
│
└── Negative signals
    ├── Hide post
    ├── Unfollow author
    ├── Mark as spam
    └── Report content
```

### The Implicit Feedback Challenge

| What You Observe | What It Might Mean | What It Might NOT Mean |
|------------------|-------------------|------------------------|
| User clicked | Interested in content | Actually liked it |
| User didn't click | Not interested | Didn't see it (scrolled past) |
| User watched 10s | Engaged with video | Fell asleep, phone in pocket |
| User liked | Positive reaction | Social obligation, habit |
| User hid post | Negative reaction | Saw it too many times |

**Key insight:** Every implicit label has false positives and false negatives.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Demonstrate the noise in implicit labels

np.random.seed(42)
n_items = 1000

# True relevance (what we wish we knew)
true_relevance = np.random.uniform(0, 1, n_items)

# Different label types with different noise characteristics
def generate_implicit_label(true_relevance, noise_level, threshold=0.5):
    """Generate noisy binary label from true relevance."""
    noisy_signal = true_relevance + np.random.normal(0, noise_level, len(true_relevance))
    return (noisy_signal > threshold).astype(int)

labels = pd.DataFrame({
    'true_relevance': true_relevance,
    'click': generate_implicit_label(true_relevance, noise_level=0.3, threshold=0.3),  # High noise, low threshold
    'like': generate_implicit_label(true_relevance, noise_level=0.2, threshold=0.5),   # Medium noise
    'share': generate_implicit_label(true_relevance, noise_level=0.15, threshold=0.7), # Low noise, high threshold
    'purchase': generate_implicit_label(true_relevance, noise_level=0.1, threshold=0.8), # Very low noise, very high threshold
})

# Show correlation with true relevance
print("Correlation of different signals with TRUE relevance:\n")
for col in ['click', 'like', 'share', 'purchase']:
    corr = np.corrcoef(labels['true_relevance'], labels[col])[0, 1]
    volume = labels[col].mean()
    print(f"  {col:12s}: correlation = {corr:.3f}, volume = {volume:.1%}")

print("\n→ Stronger signals have higher correlation but lower volume")

In [ ]:
# Visualize the signal-volume tradeoff

fig, axes = plt.subplots(1, 4, figsize=(14, 3))

for ax, signal in zip(axes, ['click', 'like', 'share', 'purchase']):
    positives = labels[labels[signal] == 1]['true_relevance']
    negatives = labels[labels[signal] == 0]['true_relevance']
    
    ax.hist(negatives, bins=20, alpha=0.5, label=f'No {signal}', density=True)
    ax.hist(positives, bins=20, alpha=0.5, label=f'{signal}', density=True)
    ax.set_xlabel('True Relevance')
    ax.set_ylabel('Density')
    ax.set_title(f'{signal.capitalize()} Signal\n(volume: {labels[signal].mean():.1%})')
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

print("Observation: Stronger signals (share, purchase) separate relevant from irrelevant better,")
print("but they're much rarer, leading to class imbalance challenges.")

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Explains the tradeoff between signal quality and volume. Mentions that clicks are noisy but abundant, while shares are clean but rare.
>
> **E6 Answer Adds:** Discusses multi-task learning to leverage both—train a shared representation on abundant clicks, then specialize for sparse shares. Mentions transfer learning from dense to sparse tasks.
>
> **Common Pitfall:** Using only one signal without discussing its limitations.

## 2. The Class Imbalance Problem

At Meta scale, positive actions are **extremely rare**:

| Action | Typical Rate | At 1B impressions/day |
|--------|--------------|----------------------|
| Click | 1-5% | 10-50M clicks |
| Like | 0.1-1% | 1-10M likes |
| Comment | 0.01-0.1% | 100K-1M comments |
| Share | 0.001-0.01% | 10K-100K shares |
| Purchase | 0.0001-0.001% | 1K-10K purchases |

Even with 1 billion impressions, you might only have 10K purchases. This creates:

1. **Modeling challenges:** Model can achieve 99.9% accuracy by predicting "no purchase" always
2. **Training challenges:** Rare events may not be sampled enough
3. **Evaluation challenges:** Standard metrics may be misleading

In [ ]:
# Demonstration: Class imbalance effects

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

np.random.seed(42)

# Generate imbalanced data
n_samples = 100000
positive_rate = 0.001  # 0.1% positive rate

# Features
X = np.random.randn(n_samples, 5)

# True labels (very imbalanced)
# Positive class has slightly different feature distribution
logits = 0.5 * X[:, 0] + 0.3 * X[:, 1] - 5  # -5 shifts the base rate to be very low
probs = 1 / (1 + np.exp(-logits))
y = np.random.binomial(1, probs)

print(f"Dataset: {n_samples:,} samples, {y.sum():,} positives ({y.mean():.2%})")

# Train a model
model = LogisticRegression(max_iter=1000)
model.fit(X, y)
y_pred = model.predict(X)
y_prob = model.predict_proba(X)[:, 1]

# Evaluate with different metrics
print("\nModel Performance:")
print(f"  Accuracy:  {accuracy_score(y, y_pred):.4f}  ← Misleading! Just predicting 0 gives {1 - y.mean():.4f}")
print(f"  Precision: {precision_score(y, y_pred, zero_division=0):.4f}")
print(f"  Recall:    {recall_score(y, y_pred):.4f}")
print(f"  F1:        {f1_score(y, y_pred):.4f}")
print(f"  AUC-ROC:   {roc_auc_score(y, y_prob):.4f}  ← Better for ranking")

# Baseline: predict all zeros
y_baseline = np.zeros_like(y)
print(f"\n  Baseline (all zeros) accuracy: {accuracy_score(y, y_baseline):.4f}")

### Handling Class Imbalance

| Technique | How It Works | When to Use |
|-----------|-------------|-------------|
| **Negative downsampling** | Sample only 1-10% of negatives | Training efficiency, memory |
| **Positive upsampling** | Repeat positives multiple times | Boost rare class signal |
| **Class weights** | Weight positive samples higher in loss | When data fits in memory |
| **Focal loss** | Down-weight easy negatives | When many "easy" negatives |
| **Calibration** | Adjust predictions post-training | When you need true probabilities |

**Meta's typical approach:** Negative downsampling during training (for efficiency), then calibration at serving time.

In [ ]:
# Demonstration: Negative downsampling with calibration

def downsample_negatives(X, y, negative_sample_rate=0.1):
    """Keep all positives, sample fraction of negatives."""
    pos_mask = y == 1
    neg_mask = y == 0
    
    # Keep all positives
    X_pos, y_pos = X[pos_mask], y[pos_mask]
    
    # Sample negatives
    n_neg_samples = int(neg_mask.sum() * negative_sample_rate)
    neg_indices = np.random.choice(np.where(neg_mask)[0], n_neg_samples, replace=False)
    X_neg, y_neg = X[neg_indices], y[neg_indices]
    
    # Combine
    X_sampled = np.vstack([X_pos, X_neg])
    y_sampled = np.hstack([y_pos, y_neg])
    
    # Shuffle
    shuffle_idx = np.random.permutation(len(y_sampled))
    return X_sampled[shuffle_idx], y_sampled[shuffle_idx], negative_sample_rate

def calibrate_predictions(predictions, sample_rate):
    """
    Calibrate predictions trained on downsampled data.
    
    If we sampled negatives at rate r, the model sees
    P'(pos) = P(pos) / (P(pos) + r * P(neg))
    
    To recover true probability:
    P(pos) = r * P'(pos) / (r * P'(pos) + (1 - P'(pos)))
    """
    r = sample_rate
    p_prime = predictions
    return (r * p_prime) / (r * p_prime + (1 - p_prime))

# Downsample and train
X_ds, y_ds, sample_rate = downsample_negatives(X, y, negative_sample_rate=0.1)
print(f"Downsampled data: {len(y_ds):,} samples, {y_ds.sum():,} positives ({y_ds.mean():.1%})")

model_ds = LogisticRegression(max_iter=1000)
model_ds.fit(X_ds, y_ds)

# Raw predictions (before calibration)
y_prob_raw = model_ds.predict_proba(X)[:, 1]

# Calibrated predictions
y_prob_cal = calibrate_predictions(y_prob_raw, sample_rate)

# Compare
print(f"\nTrue positive rate: {y.mean():.4f}")
print(f"Avg raw prediction: {y_prob_raw.mean():.4f} (biased high due to downsampling)")
print(f"Avg calibrated:     {y_prob_cal.mean():.4f} (close to true rate)")
print(f"\nAUC-ROC is unchanged: {roc_auc_score(y, y_prob_raw):.4f} (ranking preserved)")

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Explains why class imbalance matters and proposes downsampling or class weights.
>
> **E6 Answer Adds:** Discusses calibration—why raw predictions from downsampled training are biased and how to correct. Mentions that ranking metrics (AUC) are preserved but probability estimates need adjustment.
>
> **Common Pitfall:** Reporting accuracy as the main metric for imbalanced problems.

## 3. Label Delay and Attribution

At Meta, actions happen at different times after impression:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                              LABEL DELAY SPECTRUM                            │
└─────────────────────────────────────────────────────────────────────────────┘

Impression     Click      View      Like      Comment    Purchase    Return
    │            │         │          │           │          │          │
    ▼            ▼         ▼          ▼           ▼          ▼          ▼
────┬────────────┬─────────┬──────────┬───────────┬──────────┬──────────┬────▶ Time
   T=0        T+1s      T+10s      T+30s      T+5min     T+1day    T+30days
    │            │         │          │           │          │          │
    │     Immediate    Short-term    Medium      Delayed   Very delayed
    │       labels       labels      labels      labels      labels
```

### The Attribution Window Problem

**Question:** If a user sees an ad at T=0 and purchases at T=7 days, did the ad cause the purchase?

Options:
1. **Last-click attribution:** Only credit if purchase within 1 hour
2. **7-day window:** Credit if purchase within 7 days
3. **View-through:** Credit even if user didn't click but saw the ad

Each choice creates different labels and different model behavior!

In [ ]:
# Demonstration: How attribution window affects labels

np.random.seed(42)
n_impressions = 10000

# Simulate impressions with delayed conversions
impressions = pd.DataFrame({
    'impression_id': range(n_impressions),
    'user_id': np.random.randint(0, 1000, n_impressions),
    'ad_quality': np.random.uniform(0, 1, n_impressions),  # True quality of ad
})

# Conversion probability depends on ad quality, but timing is random
conversion_prob = impressions['ad_quality'] * 0.05  # Up to 5% conversion rate
will_convert = np.random.binomial(1, conversion_prob)

# Time to conversion (if converted): mix of immediate and delayed
# 30% convert within 1 hour, 50% within 1 day, 20% take up to 7 days
conversion_time = np.where(
    will_convert == 1,
    np.random.choice(
        [0.5, 12, 72, 168],  # hours: 30min, 12hr, 3day, 7day
        n_impressions,
        p=[0.3, 0.2, 0.3, 0.2]
    ),
    np.inf  # No conversion
)

impressions['will_convert'] = will_convert
impressions['conversion_time_hours'] = conversion_time

# Create labels with different attribution windows
def apply_attribution_window(df, window_hours):
    return ((df['will_convert'] == 1) & (df['conversion_time_hours'] <= window_hours)).astype(int)

impressions['label_1h'] = apply_attribution_window(impressions, 1)
impressions['label_24h'] = apply_attribution_window(impressions, 24)
impressions['label_7d'] = apply_attribution_window(impressions, 168)

print("Conversion rates by attribution window:")
print(f"  1-hour window:  {impressions['label_1h'].mean():.2%}")
print(f"  24-hour window: {impressions['label_24h'].mean():.2%}")
print(f"  7-day window:   {impressions['label_7d'].mean():.2%}")

# Train models with different labels
from sklearn.linear_model import LogisticRegression

X = impressions[['ad_quality']].values

print("\nModel correlation with TRUE ad quality:")
for label_col, window in [('label_1h', '1-hour'), ('label_24h', '24-hour'), ('label_7d', '7-day')]:
    model = LogisticRegression()
    model.fit(X, impressions[label_col])
    coef = model.coef_[0][0]
    print(f"  {window} model coefficient: {coef:.3f}")
    
print("\n→ Wider window captures more true signal (higher coefficient on ad_quality)")

### Training with Delayed Labels

**Problem:** If labels take 7 days, training data is always 7+ days old.

**Solutions:**
1. **Multi-task learning:** Train on immediate signals (clicks), transfer to delayed signals (purchases)
2. **Label propagation:** Use partial labels (label at T+1 day as proxy for T+7 day)
3. **Survival modeling:** Model time-to-conversion, not just binary conversion
4. **Fresh feature, stale label:** Use fresh features at serving, accept stale labels at training

## 4. Label Bias: Systematic Errors in Labels

### Types of Label Bias

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                              LABEL BIAS TAXONOMY                             │
└─────────────────────────────────────────────────────────────────────────────┘

SELECTION BIAS
├── You only observe labels for items you showed
├── Items never shown have no labels
└── Model can't learn about unexplored space

POSITION BIAS
├── Higher positions get more clicks
├── Labels confound relevance with position
└── Model learns "position 1 is good" not "relevant is good"

POPULARITY BIAS
├── Popular items get more engagement
├── Labels confound quality with popularity
└── Model reinforces popularity, new content starves

PRESENTATION BIAS
├── Visual presentation affects engagement
├── A better thumbnail != better content
└── Model learns to predict thumbnail quality, not content quality

SOCIAL BIAS
├── Actions influenced by social proof
├── "100K likes" → more likely to like
└── Model learns to predict what's already popular
```

In [ ]:
# Demonstration: Popularity bias creating rich-get-richer dynamics

def simulate_popularity_bias(n_items=100, n_rounds=50, bias_strength=0.5):
    """
    Simulate how popularity-biased labels create rich-get-richer dynamics.
    
    Items have true quality, but observed engagement also depends on current popularity.
    """
    np.random.seed(42)
    
    # True quality (fixed)
    true_quality = np.random.uniform(0, 1, n_items)
    
    # Current popularity (changes over time)
    popularity = np.ones(n_items) / n_items  # Start equal
    
    history = []
    
    for round_num in range(n_rounds):
        # Observed engagement = quality + bias * popularity
        observed_score = (1 - bias_strength) * true_quality + bias_strength * popularity
        
        # Show top items (greedy)
        shown_items = np.argsort(observed_score)[-10:]  # Show top 10
        
        # Update popularity based on who was shown
        new_popularity = np.zeros(n_items)
        for item in shown_items:
            new_popularity[item] = true_quality[item]  # Actual engagement
        new_popularity = new_popularity / new_popularity.sum()
        
        # Popularity accumulates over time
        popularity = 0.9 * popularity + 0.1 * new_popularity
        popularity = popularity / popularity.sum()
        
        # Track diversity: how many unique items have been shown
        history.append({
            'round': round_num,
            'top_item_quality': true_quality[shown_items].mean(),
            'quality_correlation': np.corrcoef(true_quality, observed_score)[0, 1],
            'gini_coefficient': gini(popularity)
        })
    
    return pd.DataFrame(history), true_quality, popularity

def gini(x):
    """Calculate Gini coefficient (0 = equal, 1 = one item has everything)"""
    x = np.sort(x)
    n = len(x)
    index = np.arange(1, n + 1)
    return (2 * np.sum(index * x) - (n + 1) * np.sum(x)) / (n * np.sum(x))

# Compare low vs high bias
history_low, q_low, p_low = simulate_popularity_bias(bias_strength=0.1)
history_high, q_high, p_high = simulate_popularity_bias(bias_strength=0.7)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Gini coefficient over time
axes[0].plot(history_low['round'], history_low['gini_coefficient'], label='Low bias (0.1)')
axes[0].plot(history_high['round'], history_high['gini_coefficient'], label='High bias (0.7)')
axes[0].set_xlabel('Round')
axes[0].set_ylabel('Gini Coefficient (inequality)')
axes[0].set_title('Popularity Concentration Over Time')
axes[0].legend()

# Final popularity vs true quality
axes[1].scatter(q_high, p_high, alpha=0.5)
axes[1].set_xlabel('True Quality')
axes[1].set_ylabel('Final Popularity Share')
axes[1].set_title('High Bias: Popularity Disconnected from Quality')

plt.tight_layout()
plt.show()

print(f"Correlation (quality, final popularity):")
print(f"  Low bias:  {np.corrcoef(q_low, p_low)[0,1]:.3f}")
print(f"  High bias: {np.corrcoef(q_high, p_high)[0,1]:.3f}")

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Identifies popularity bias and proposes exploration (random traffic, Thompson sampling) to counteract it.
>
> **E6 Answer Adds:** Discusses systemic solutions—inverse propensity weighting for training, diversity constraints in ranking, cold start strategies for new content, and monitoring for concentration metrics.
>
> **Common Pitfall:** Training on biased labels without acknowledging the bias exists.

## 5. Data Collection Strategies

### Logging for ML

At Meta, logging is designed for ML from the start:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                          ML LOGGING REQUIREMENTS                             │
└─────────────────────────────────────────────────────────────────────────────┘

For every impression, log:
├── Request context
│   ├── User ID (or anonymous ID)
│   ├── Session ID
│   ├── Timestamp
│   ├── Device type
│   └── Geo location (bucketed for privacy)
│
├── Candidate information
│   ├── Item ID
│   ├── Item features (at request time)
│   ├── Position in ranking
│   └── Model score
│
├── User features (at request time)
│   ├── Historical features
│   ├── Real-time features
│   └── Session features
│
└── Outcomes (joined later)
    ├── Click (immediate)
    ├── Dwell time
    ├── Engagement (like, comment, share)
    ├── Negative actions (hide, report)
    └── Downstream conversions
```

### Why Log Features at Request Time?

**Problem:** If you log only user ID and item ID, and join features later, you get **label-feature leakage**.

Example:
- User clicks ad at T=0
- At T+1 day, user's "click history" feature includes this click
- If you join features at T+7 days, the "click history" feature now includes the click you're trying to predict

**Solution:** Log features at request time, so training uses the same features that were available at serving time.

In [ ]:
# Demonstration: Label leakage through feature joins

np.random.seed(42)
n_events = 1000

# Simulate events over time
events = pd.DataFrame({
    'event_id': range(n_events),
    'user_id': np.random.randint(0, 100, n_events),
    'timestamp': pd.date_range('2024-01-01', periods=n_events, freq='1min'),
    'true_intent': np.random.uniform(0, 1, n_events),  # User's true click propensity
})

# Labels (click or not)
events['clicked'] = (np.random.random(n_events) < events['true_intent']).astype(int)

# CORRECT: Feature logged at request time (doesn't include current event)
def compute_historical_ctr_correct(df):
    """Compute CTR based on events BEFORE current event."""
    result = []
    for idx, row in df.iterrows():
        user_history = df[(df['user_id'] == row['user_id']) & (df['timestamp'] < row['timestamp'])]
        if len(user_history) > 0:
            result.append(user_history['clicked'].mean())
        else:
            result.append(0.5)  # Default for new users
    return result

# WRONG: Feature joined later (includes current event - LEAKAGE)
def compute_historical_ctr_leaky(df):
    """Compute CTR INCLUDING current event - causes leakage."""
    result = []
    for idx, row in df.iterrows():
        user_history = df[df['user_id'] == row['user_id']]  # Includes current!
        result.append(user_history['clicked'].mean())
    return result

events['hist_ctr_correct'] = compute_historical_ctr_correct(events)
events['hist_ctr_leaky'] = compute_historical_ctr_leaky(events)

# Train models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_correct = events[['hist_ctr_correct']].values
X_leaky = events[['hist_ctr_leaky']].values
y = events['clicked'].values

# Evaluate
print("Correlation of feature with label:")
print(f"  Correct (no leakage): {np.corrcoef(X_correct.flatten(), y)[0,1]:.3f}")
print(f"  Leaky:                {np.corrcoef(X_leaky.flatten(), y)[0,1]:.3f}  ← Artificially high!")
print("\n→ Leaky feature has artificially high correlation because it includes the label!")

## 6. Negative Sampling for Multi-Class Problems

For candidate generation models, you need to predict which of millions of items a user will engage with.

**Problem:** You can't train on all negatives (too many).

**Solution:** Sample negatives, but how you sample affects what the model learns.

| Sampling Strategy | What Model Learns | Best For |
|-------------------|-------------------|----------|
| **Uniform random** | Distinguish target from random | General retrieval |
| **Popularity-based** | Distinguish target from popular items | When popular ≠ relevant |
| **In-batch** | Distinguish target from other users' targets | Efficient training |
| **Hard negatives** | Distinguish target from similar items | Fine-grained ranking |

In [ ]:
# Demonstration: Impact of negative sampling strategy

np.random.seed(42)
n_items = 1000
n_samples = 5000

# Item properties
item_popularity = np.random.pareto(2, n_items)  # Power law popularity
item_popularity = item_popularity / item_popularity.sum()
item_quality = np.random.uniform(0, 1, n_items)

# Generate positive interactions (user likes item)
# Positive = high quality items (with some noise)
positive_probs = item_quality / item_quality.sum()
positives = np.random.choice(n_items, n_samples, p=positive_probs)

# Different negative sampling strategies
def sample_negatives_uniform(positives, n_items):
    return np.random.randint(0, n_items, len(positives))

def sample_negatives_popularity(positives, n_items, popularity):
    return np.random.choice(n_items, len(positives), p=popularity)

def sample_negatives_hard(positives, n_items, embeddings, k=5):
    """Sample from items similar to positives (hard negatives)."""
    negatives = []
    for pos in positives:
        # Find k nearest neighbors, sample one
        distances = np.abs(embeddings - embeddings[pos])
        nearest = np.argsort(distances)[1:k+1]  # Exclude self
        negatives.append(np.random.choice(nearest))
    return np.array(negatives)

# Simple 1D embeddings based on quality (for demonstration)
embeddings = item_quality

neg_uniform = sample_negatives_uniform(positives, n_items)
neg_popular = sample_negatives_popularity(positives, n_items, item_popularity)
neg_hard = sample_negatives_hard(positives, n_items, embeddings)

# Analyze what each strategy surfaces
print("Average item QUALITY in negative samples:")
print(f"  Uniform:    {item_quality[neg_uniform].mean():.3f}")
print(f"  Popularity: {item_quality[neg_popular].mean():.3f}")
print(f"  Hard:       {item_quality[neg_hard].mean():.3f}")

print("\nAverage item POPULARITY in negative samples:")
print(f"  Uniform:    {item_popularity[neg_uniform].mean():.6f}")
print(f"  Popularity: {item_popularity[neg_popular].mean():.6f}")
print(f"  Hard:       {item_popularity[neg_hard].mean():.6f}")

print("\n→ Hard negatives have higher quality (closer to positives)")
print("  This forces the model to learn fine-grained distinctions")

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Explains why negative sampling is necessary and describes uniform vs popularity-based sampling.
>
> **E6 Answer Adds:** Discusses hard negative mining, curriculum learning (start easy, get harder), and how sampling distribution affects what the model learns. Mentions potential issues with in-batch negatives (batch composition matters).
>
> **Common Pitfall:** Not mentioning that negative sampling strategy affects model behavior.

---

## 🎯 Interview Drills

### Drill 1: "Where do your labels come from for [X system]?"

**Framework:**
1. List explicit signals (if any)
2. List implicit signals with their noise characteristics
3. Describe positive vs negative labels
4. Mention biases in labels
5. Describe attribution window

**Example for Ad Click Prediction:**
> "For ad click prediction, labels come from click events. This is an implicit signal—a click means interest but not necessarily satisfaction. Positive labels are clicks, negative labels are non-clicks. Key biases: position bias (higher ads get more clicks), presentation bias (better creative gets more clicks regardless of relevance). For conversion-based models, we use 7-day attribution windows, which means training data is always at least 7 days stale."

---

### Drill 2: "How do you handle a 0.01% positive rate?"

**Strong answer:**
> "With 0.01% positive rate, we face severe class imbalance. Solutions: (1) Negative downsampling—keep all positives, sample 1-10% of negatives, then calibrate predictions at serving. (2) Use ranking metrics like AUC that are robust to imbalance instead of accuracy. (3) Multi-task learning—train on a related but more frequent signal (clicks) and transfer to the rare signal (purchases). (4) For evaluation, use stratified sampling to ensure enough positives in test set."

---

### Drill 3: "How do you prevent label leakage?"

**Strong answer:**
> "Label leakage happens when features contain information about the label that wouldn't be available at serving time. Prevention: (1) Log features at request time, not join later. (2) Use point-in-time feature snapshots for historical features. (3) For user history features, ensure they don't include the current event. (4) Audit features by checking correlation with label—suspiciously high correlation suggests leakage. (5) Validate by checking that offline and online metrics agree—large discrepancy suggests training-serving skew or leakage."

---

### Drill 4: "What breaks at Meta scale with this data approach?"

**Strong answer:**
> "At Meta scale: (1) Logging volume—billions of impressions per day, storage becomes expensive, need sampling or aggregation. (2) Feature joins—joining user features to impression logs at scale requires distributed systems, and stale joins introduce training-serving skew. (3) Label delays—waiting 7 days for labels means training data is always stale, model can't adapt to trends. (4) Negative sampling—with millions of items, can't enumerate all negatives, sampling strategy significantly affects model behavior. (5) Feedback loops—model predictions determine what gets shown, which determines future labels, can cause drift."

---

## Summary: Data & Labels Checklist

When designing an ML system, verify:

- [ ] **Label source identified:** Explicit or implicit? What's the noise level?
- [ ] **Class imbalance addressed:** What's the positive rate? Downsampling? Calibration?
- [ ] **Label delay understood:** How long until labels are available? Impact on freshness?
- [ ] **Attribution window defined:** For delayed outcomes, what window makes sense?
- [ ] **Biases acknowledged:** Position? Popularity? Selection? Presentation?
- [ ] **Logging at request time:** Are features logged when prediction is made?
- [ ] **Negative sampling strategy:** Uniform? Popularity? Hard negatives?
- [ ] **Label leakage prevented:** No future information in features?
- [ ] **Label quality validated:** Spot checks? Human review? Correlation with ground truth?